# Tutorial notebook for LCA-Modeller

Welcome to the *LCA-Modeller* library tutorial! *LCA-Modeller* offers a streamlined interface designed to facilitate the creation of **parametric LCA models** with **prospective capabilities**. It builds on the open-source libraries [*lca-algebraic*](https://lca-algebraic.readthedocs.io/) and [*premise*](https://premise.readthedocs.io/), so having a basic understanding of these tools, while not essential, is recommended.

The core functionality of *LCA-Modeller* revolves around reading a user-provided configuration file that defines the LCA model. From this configuration, *LCA-Modeller* generates a parametric LCA model, which can be evaluated for any parameter values using *lca-algebraic*'s built-in functions.
If prospective scenarios are provided, *premise* is used to adapt the EcoInvent database to future conditions. The parametric LCA model then selects the appropriate prospective databases based on the user-specified evaluation year.

In [1]:
# Import libraries
from lca_modeller.io.configuration import LCAProblemConfigurator
import lca_algebraic as agb
import brightway2 as bw
import numpy as np

# Add data folder to path
import sys
import os.path as pth
sys.path.insert(0, '../..')
DATA_FOLDER = './data'

## 1. A basic example

In [2]:
# Provide the configuration file where the ecoinvent database, system model and impact categories are defined
configuration_file = pth.join(DATA_FOLDER, 'basic_example.yaml')
project, model, methods = LCAProblemConfigurator(configuration_file).generate()

Initial setup of EcoInvent already done, skipping. To reset the project use option `reset_project=True` in configuration file.
Building LCA model from configuration file
LCA model successfully created


In [3]:
# Provide values for parameters
parameters = {
    'a': 1.0,
    'b': 0.7,
    'c': 1.2,
    'd': 10.0,
}

# Compute impacts. The first call takes some time (compilation of the LCA functions) but next ones are much faster.
df = agb.compute_impacts(
    model,
    methods,
    **parameters
)
df

,climate change - global warming potential (GWP100)[kg CO2-Eq],total: ecosystem quality - ecosystem quality[species.yr],total: human health - human health[DALYs],total: natural resources - natural resources[USD 2013]
model,89.6781,4.08824e-06,0.000217808,3.95525


In [4]:
# Compute impacts for several values at once
parameters = {
    'a': [1.0, 1.2, 1.4],  # list of values
    'b': [0.7, 0.5, 0.2],  # all lists must be of the same size
    'c': 1.2,  # constant value
    'd': 10.0,
}
agb.compute_impacts(
    model,
    methods,
    **parameters
)

,,climate change - global warming potential (GWP100)[kg CO2-Eq],total: ecosystem quality - ecosystem quality[species.yr],total: human health - human health[DALYs],total: natural resources - natural resources[USD 2013]
a,b,,,,
1,0.7,89.6781,4.08824e-06,0.000217808,3.95525
1.2,0.5,87.7572,4.00675e-06,0.000213212,3.86261
1.4,0.2,68.5128,3.12332e-06,0.000166465,3.02178


In [8]:
# Another example
parameters = {
    'a': np.linspace(1.0, 2.0, 10),
    'b': np.linspace(0.7, 0.0, 10),
    'c': 1.2,
    'd': 10.0,
}
agb.compute_impacts(
    model,
    methods,
    **parameters
)

,climate change - global warming potential (GWP100)[kg CO2-Eq],total: ecosystem quality - ecosystem quality[species.yr],total: human health - human health[DALYs],total: natural resources - natural resources[USD 2013]
model,89.6781,4.08824e-06,0.000217808,3.95525
1,92.311,4.21413e-06,0.000224257,4.06375
2,93.6146,4.27782e-06,0.000227465,4.1157
3,93.5787,4.27905e-06,0.000227411,4.11038
4,92.1966,4.21765e-06,0.000224079,4.04732
5,89.4633,4.0935e-06,0.000217458,3.92618
6,85.3753,3.9065e-06,0.00020754,3.7467
7,79.9297,3.65658e-06,0.000194318,3.50868
8,73.1247,3.3437e-06,0.000177788,3.21199
9,64.9585,2.96782e-06,0.000157947,2.8565


## 2. Get impacts by phase or subsystem

In [14]:
# Provide the configuration file where the ecoinvent database, system model and impact categories are defined
configuration_file = pth.join(DATA_FOLDER, 'basic_example_split_impacts.yaml')
project, model, methods = LCAProblemConfigurator(configuration_file).generate()

Initial setup of EcoInvent already done, skipping. To reset the project use option `reset_project=True` in configuration file.
Building LCA model from configuration file
LCA model successfully created


In [16]:
# Provide values for parameters
parameters = {
    'a': 1.0,
    'b': 0.7,
    'c': 1.2,
    'd': 10.0,
}

# Compute impacts and provide results by subsystem 
agb.compute_impacts(
    model,
    methods,
    axis='subsystem',
    **parameters
)

,climate change - global warming potential (GWP100)[kg CO2-Eq],total: ecosystem quality - ecosystem quality[species.yr],total: human health - human health[DALYs],total: natural resources - natural resources[USD 2013]
subsystem,,,,
_other_,3.6e-05,1.01003e-13,3.34e-11,0
drink,2.40278,6.09026e-08,5.45695e-06,0.169253
food,87.2753,4.02734e-06,0.000212351,3.786
*sum*,89.6781,4.08824e-06,0.000217808,3.95525


In [17]:
# Compute impacts and provide results by food type 
agb.compute_impacts(
    model,
    methods,
    axis='foodtype',
    **parameters
)

,climate change - global warming potential (GWP100)[kg CO2-Eq],total: ecosystem quality - ecosystem quality[species.yr],total: human health - human health[DALYs],total: natural resources - natural resources[USD 2013]
foodtype,,,,
_other_,3.6e-05,1.01003e-13,3.34e-11,0
carrots,0.137479,4.41999e-09,4.18723e-07,0.00843005
cheese,87.1378,4.02292e-06,0.000211932,3.77757
soybean,2.40278,6.09026e-08,5.45695e-06,0.169253
*sum*,89.6781,4.08824e-06,0.000217808,3.95525


## 3. Prospective LCA

Warning: depending on the IAM, the locations might be changed compared to the original ecoinvent database.

In [8]:
# Provide the configuration file where the ecoinvent database, system model and impact categories are defined
configuration_file = pth.join(DATA_FOLDER, 'prospective_lca.yaml')
project, model, methods = LCAProblemConfigurator(configuration_file).generate()

Initial setup of EcoInvent already done, skipping. To reset the project use option `reset_project=True` in configuration file.
Building LCA model from configuration file
LCA model successfully created


In [9]:
# List databases installed in the project
bw.databases

Databases dictionary with 8 object(s):
	Foreground DB
	MyForeground
	biosphere3
	biosphere_user
	ecoinvent-3.10-cutoff
	ecoinvent_cutoff_3.10_remind_SSP2-Base_2020
	ecoinvent_cutoff_3.10_remind_SSP2-Base_2030
	ecoinvent_cutoff_3.10_remind_SSP2-Base_2040

In [10]:
# Running a project with premise will automatically add the parameter 'year' to the model to enable the user to select the year of evaluation.
parameters = {
    'year': [2020, 2025, 2030, 2035, 2040],  # if the year falls between two ecoinvent datasets (e.g. 2020 and 2030), the result will be a linear interpolation of the two
    'a': [1.0, 1.2, 1.4, 1.6, 1.8],
    'b': [0.7, 0.5, 0.2, 0.1, 0.0],
    'c': 1.2,
    'd': 10.0,
}
agb.compute_impacts(
    model,
    methods,
    **parameters
)

,,,climate change - global warming potential (GWP100)[kg CO2-Eq],total: ecosystem quality - ecosystem quality[species.yr],total: human health - human health[DALYs],total: natural resources - natural resources[USD 2013]
year,a,b,,,,
2020,1,0.7,88.2462,4.07912e-06,0.000204248,3.74304
2025,1.2,0.5,87.058,4.0023e-06,0.000206581,3.75886
2030,1.4,0.2,68.5128,3.12332e-06,0.000166465,3.02178
2035,1.6,0.1,65.1766,2.97297e-06,0.000158399,2.87236
2040,1.8,0,58.7151,2.67743e-06,0.000142725,2.58863


## 4. Advanced examples

### Reference a previously defined activity and modify existing activities

In [11]:
# Provide the configuration file where the ecoinvent database, system model and impact categories are defined
configuration_file = pth.join(DATA_FOLDER, 'advanced_example.yaml')
project, model, methods = LCAProblemConfigurator(configuration_file).generate()

Initial setup of EcoInvent already done, skipping. To reset the project use option `reset_project=True` in configuration file.
Building LCA model from configuration file
LCA model successfully created


In [12]:
# Running a project with premise will automatically add the parameter 'year' to the model to enable the user to select the year of evaluation.
parameters = {
    'year': [2020, 2025, 2030, 2035, 2040],  # if the year falls between two ecoinvent datasets (e.g. 2020 and 2030), the result will be a linear interpolation of the two
    'a': [1.0, 1.2, 1.4, 1.6, 1.8],
    'b': [0.7, 0.5, 0.2, 0.1, 0.0],
    'c': 1.2,
    'd': 10.0,
    'p': [0.3, 0.2, 0.15, 0.12, 0.08]
}
agb.compute_impacts(
    model,
    methods,
    **parameters
)

,,,,climate change - global warming potential (GWP100)[kg CO2-Eq],total: ecosystem quality - ecosystem quality[species.yr],total: human health - human health[DALYs],total: natural resources - natural resources[USD 2013]
year,a,b,p,,,,
2020,1,0.7,0.3,431.872,2.01535e-05,0.00100108,18.0676
2025,1.2,0.5,0.2,356.375,1.65131e-05,0.000846659,15.2176
2030,1.4,0.2,0.15,240.121,1.10388e-05,0.000584144,10.4708
2035,1.6,0.1,0.12,200.318,9.20526e-06,0.000487368,8.73998
2040,1.8,0,0.08,160.558,7.37282e-06,0.000390691,7.01224


## 5. Define your own LCIA methods

In [13]:
# Provide the configuration file where the ecoinvent database, system model and impact categories are defined
configuration_file = pth.join(DATA_FOLDER, 'custom_lcia_methods.yaml')
project, model, methods = LCAProblemConfigurator(configuration_file).generate()

Writing activities to SQLite3 database:


Initial setup of EcoInvent already done, skipping. To reset the project use option `reset_project=True` in configuration file.
Creating custom LCIA method  ('Custom methods', 'climate change', 'GWP100 (updated)')


0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 10/21/2025 14:33:28
  Finished: 10/21/2025 14:33:28
  Total time elapsed: 00:00:00
  CPU %: 2.50
  Memory %: 5.90
Added 1 new biosphere flows
1 methods
130 cfs
0 unlinked cfs
Wrote 1 LCIA methods with 130 characterization factors
Wrote matching file to:
/Users/f.pollet/POSTDOC/5 - Tools/LCA-Modeller/lca_modeller/notebook/data/lcia_gwp100_custom_updated.xlsx
Building LCA model from configuration file
LCA model successfully created


In [14]:
# The custom method is automatically added to the list
methods

[('ReCiPe 2016 v1.03, midpoint (H)',
  'climate change',
  'global warming potential (GWP100)'),
 ('Custom methods', 'climate change', 'GWP100 (updated)')]

In [15]:
# Compute impacts with new method
parameters = {
    'a': [1.0, 1.2, 1.4, 1.6, 1.8],
    'b': [0.7, 0.5, 0.2, 0.1, 0.0],
    'c': 1.2,
    'd': 10.0,
}

agb.compute_impacts(
    model,
    methods,
    **parameters
)

,,climate change - global warming potential (GWP100)[kg CO2-Eq],climate change - GWP100 (updated)[kg CO2-Eq]
a,b,,
1,0.7,89.6781,123.057
1.2,0.5,87.7572,121.124
1.4,0.2,68.5128,101.802
1.6,0.1,65.1766,98.4502
1.8,0,58.7151,91.9618


# 6. Declare metadata for parameters
For example, declare min/max values and distribution type for each parameter to run a Monte Carlo using lca-algebraic's capabilities

In [20]:
# Provide the configuration file where the ecoinvent database, system model and impact categories are defined
configuration_file = pth.join(DATA_FOLDER, 'metadata_example.yaml')
project, model, methods = LCAProblemConfigurator(configuration_file).generate()

Initial setup of EcoInvent already done, skipping. To reset the project use option `reset_project=True` in configuration file.
Building LCA model from configuration file
LCA model successfully created


In [21]:
agb.list_parameters()

group,name,label,default,min,max,std,distrib,unit,db
,a,a,2,1,3,,linear,m,Foreground DB
,b,b,2,1,3,,triangle,kg,Foreground DB
,c,c,0.1,0,1,,fixed,kg,Foreground DB
,d,d,13.2,0,1,,fixed,,Foreground DB


In [22]:
agb.compute_impacts(model, methods)

,climate change - global warming potential (GWP100)[kg CO2-Eq],total: ecosystem quality - ecosystem quality[species.yr],total: human health - human health[DALYs],total: natural resources - natural resources[USD 2013]
model,324.081,1.49235e-05,0.000788116,14.0994


In [19]:
agb.incer_stochastic_dashboard(model, methods, n=1024)

Generating samples ...
Transforming samples ...
Processing Sobol indices ...
Processing sobol for ('ReCiPe 2016 v1.03, midpoint (H)', 'climate change', 'global warming potential (GWP100)')
Processing sobol for ('ReCiPe 2016 v1.03, endpoint (H)', 'total: ecosystem quality', 'ecosystem quality')
Processing sobol for ('ReCiPe 2016 v1.03, endpoint (H)', 'total: human health', 'human health')
Processing sobol for ('ReCiPe 2016 v1.03, endpoint (H)', 'total: natural resources', 'natural resources')


/opt/miniconda3/envs/lca-modeller/lib/python3.11/site-packages/SALib/util/__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))
